In [1]:
import pandas as pd
import os
import boto3
import numpy as  np
import pandas as pd

In [2]:
# setup
os.chdir('//FS2\RTO\\CIS-PD MUSC\decoded_forms')
client = boto3.client('athena', 
                      region_name='us-west-2')
user_id_pairings = pd.read_csv("videoID.csv")
form509 = pd.read_hdf("form509.h5")
print(form509)

     SubjectCode  SiteID            VisitNm   FormDate  Q10  Q11  Q12  Q13  \
0         1003.0  1313.0    2 Weeks: Time 0 2017-07-03  2.0  0.0  0.0  1.0   
1         1003.0  1313.0   2 Weeks: Time 30 2017-07-03  1.0  0.0  0.0  0.0   
2         1003.0  1313.0   2 Weeks: Time 60 2017-07-03  1.0  2.0  0.0  0.0   
3         1003.0  1313.0   2 Weeks: Time 90 2017-07-03  0.0  0.0  0.0  0.0   
4         1003.0  1313.0  2 Weeks: Time 120 2017-07-03  1.0  0.0  0.0  1.0   
5         1003.0  1313.0  2 Weeks: Time 150 2017-07-03  1.0  1.0  0.0  0.0   
6         1003.0  1313.0            1 Month 2017-07-24  1.0  1.0  0.0  0.0   
7         1004.0  1332.0    2 Weeks: Time 0 2017-07-07  0.0  0.0  0.0  0.0   
8         1004.0  1332.0   2 Weeks: Time 30 2017-07-07  0.0  0.0  0.0  0.0   
9         1004.0  1332.0   2 Weeks: Time 60 2017-07-07  0.0  1.0  1.0  0.0   
10        1004.0  1332.0   2 Weeks: Time 90 2017-07-07  0.0  0.0  1.0  0.0   
11        1004.0  1332.0  2 Weeks: Time 120 2017-07-07  0.0  0.0

In [3]:
# get the 4 digit user_id from the corresponding 6 digit user_id
def user_id_6_to_4(user_id_6):
    for index, row in user_id_pairings.iterrows():
        if (row['Subj ID Athena']==user_id_6):
            return int(row['SubjectCode'])
    return None

In [4]:
# get the 6 digit user_id from the corresponding 4 digit user_id
def user_id_4_to_6(user_id_4):
    for index, row in user_id_pairings.iterrows():
        if (row['SubjectCode']==user_id_4):
            return int(row['Subj ID Athena'])
    return None

In [5]:
# generate query from row in form509
def generate_query(row):
    template1 = "SELECT *\nFROM watch_accelerometer\nwhere experiment = 'Clinicians'\nand user_id = "
    template2 = "\nand timestamp >= timestamp '"
    template3 = "'\nand timestamp <= timestamp '"
    template4 = "'"
    
    user_ID_4 = row['SubjectCode']
    
    user_ID_6 = user_id_4_to_6(user_ID_4).__str__()
    
    time = row['Q146_UTC']
    startTime =  (time - pd.Timedelta('00:15:00')).__str__()
    endTime = (time + pd.Timedelta('00:45:00')).__str__()
    
    if startTime == "NaT" or endTime == "NaT":
        print("invalid time")
        return None
    query = template1 + user_ID_6 + template2 + startTime + template3 + endTime + template4
    return query

In [6]:
# call query
def call_query(query):
    database='clinician_input_study'
    output_location='s3://aws-athena-query-results-arunjayaraman-us-west-2/'
    encryption_option='SSE_S3'

    query_info = client.start_query_execution(
        QueryString=query, 
        QueryExecutionContext={ 
            'Database': database
        },
        ResultConfiguration={ 
            'OutputLocation': output_location,
            'EncryptionConfiguration': { 
                'EncryptionOption': encryption_option
            }
        }
    )
    return query_info

In [7]:
# return the results of the query associated with query_info
def get_query_results(query_info):
    query_id = query_info['QueryExecutionId']

    while True:
        try:
            response = client.get_query_results(QueryExecutionId= query_id)
        except client.exceptions.InvalidRequestException:
            continue
        break
    
    results = pd.DataFrame(response)
    print(response)
    
    data = pd.DataFrame(response['ResultSet']['Rows'])
    print(data.shape)
    
    # format the results
    columns_names = pd.DataFrame(response['ResultSet']['Rows'][0]['Data'])
    cols = columns_names.get_values()
    columns = []
    for col in cols:
        columns.append(col[0])

    formatted_rows = []

    for index, row in data.iterrows():
        if index != 0:
            df = pd.DataFrame(row['Data'])
            vals = df.get_values()
            values = []
            for val in vals:
                values.append(val[0])
            formatted_rows.append(values)
    
    new_rows = np.array(formatted_rows)

       
    if (new_rows.__len__() == 0):
        print("empty data")
        return None
    
    formatted_results = pd.DataFrame(new_rows, columns=columns)

    return formatted_results

In [8]:
# get visit num from the visit information
def get_visit_num(VisitNm):
    if VisitNm.startswith("2 Weeks"):
        if VisitNm.endswith(" 0"):
            return '0'
        elif VisitNm.endswith("30"):
            return '1'
        elif VisitNm.endswith("60"):
            return '2'
        elif VisitNm.endswith("90"):
            return '3'
        elif VisitNm.endswith("120"):
            return '4'
        elif VisitNm.endswith("150"):
            return '5'
    else:
        return '6'

In [9]:
# fetch data for each row in form509
def iterate_over_form509():
    for index, row in form509.iterrows():
        id_4 = int(row['SubjectCode']).__str__()
        visit_num = get_visit_num(row['VisitNm'])
        path = "//FS2\RTO\\CIS-PD Study\MJFF Curation\ClinicVisitACC\\"
        filename = path + id_4 + "_session_" + visit_num + ".csv"
        
        query = generate_query(row)
        
        if (query is None):
            print("invalid query for subject "+ id_4 + " visit " + visit_num)
            continue
        
        query_info = call_query(query+ 'limit 100')
        query_results = get_query_results(query_info)
            
        if (query_results is None):
            print ("empty data for subject " + id_4 + " visit " + visit_num)
            continue
        
        query_results.to_csv(filename)
            

In [10]:
# run
iterate_over_form509()

NoCredentialsError: Unable to locate credentials